In [ ]:
import string
import pandas as pd
import numpy as np
import time
from itertools import combinations
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def n_grams_word(text, k):
  vectorizer = CountVectorizer(token_pattern=r'(?u)\b\w+\b', ngram_range=(k,k))
  X = vectorizer.fit_transform(text)
  return vectorizer.get_feature_names_out()

In [ ]:
def n_grams_char(text, k):
  vectorizer = CountVectorizer(analyzer="char", ngram_range=(k,k))
  X = vectorizer.fit_transform(text)
  return vectorizer.get_feature_names_out()

In [ ]:
def jaccard_similarity(text, comb):

  for c in comb:
    d_a = c[0]
    d_b = c[1]

    k_grams_char_2_a = set(n_grams_char([text[d_a]], 2))
    k_grams_char_2_b = set(n_grams_char([text[d_b]], 2))

    k_grams_char_3_a = set(n_grams_char([text[d_a]], 3))
    k_grams_char_3_b = set(n_grams_char([text[d_b]], 3))

    n_grams_word_a = set(n_grams_word([text[d_a]], 2))
    n_grams_word_b = set(n_grams_word([text[d_b]], 2))

    intersection_2_char = len(k_grams_char_2_a.intersection(k_grams_char_2_b))
    intersection_3_char = len(k_grams_char_3_a.intersection(k_grams_char_3_b))
    intersection_word = len(n_grams_word_a.intersection(n_grams_word_b))

    union_2_char = len(k_grams_char_2_a.union(k_grams_char_2_b))
    union_3_char = len(k_grams_char_3_a.union(k_grams_char_3_b))
    union_word = len(n_grams_word_a.union(n_grams_word_b))
    
    print(f"Jaccard Similarity {c} for k-gram for 2 characters: {intersection_2_char/union_2_char}")
    print(f"Jaccard Similarity {c} for k-gram for 3 characters: {intersection_3_char/union_3_char}")
    print(f"Jaccard Similarity {c} for k-gram for 2 word: {intersection_word/union_word}")

In [ ]:
def mult_hashing(x, a, i, m = 10000):
  return (x*(a+i)/2**(x % 1)) % m

In [ ]:
def min_hashing(k_gram, k):
  v = np.full((1, k), np.inf)
  for g in k_gram:
    for j in range(k):
      s = int(g.encode('utf-8').hex(),16)
      # a_lst = [10, 42, 313, 739, 852]
      # a = 0.7867
      # a = 57683
      # a = 97683
      a = 852
      # s = sha1_hashing(g,j)
      h = mult_hashing(s,a,j)
      if (h < v[0][j]):
        v[0][j] = h
  return v

In [ ]:
def jaccard_hashing(m_a, m_b, k):
  c = []
  for i, x in enumerate(m_a):
    if x == m_b[i]:
      c.append(1)
  return sum(c)/k

In [ ]:
# main
url = "https://raw.githubusercontent.com/koaning/icepickle/main/datasets/imdb_subset.csv"
df = pd.read_csv(url) # This is how you read a csv file to a pandas frame 
corpus = list(df['text'])
corpus_small = corpus[:4] # This is a list of 4 movie reviews

k_grams_word = []
k_grams_char_2 = []
k_grams_char_3 = []

for c in corpus_small:
  c_lst = [c]
  k_grams_char_2.append(n_grams_char(c_lst, 2))
  k_grams_char_3.append(n_grams_char(c_lst, 3))
  k_grams_word.append(n_grams_word(c_lst, 2))

for i, k in enumerate(k_grams_char_2):
  print(f"length 2-gram char D{i}: {len(k)}\n{k[0:10]}")
print('\n')
for i, k in enumerate(k_grams_char_3):
  print(f"length 3-gram char D{i}: {len(k)}\n{k[0:10]}")
print('\n')
for i, k in enumerate(k_grams_word):
  print(f"length 2-gram word D{i}: {len(k)}\n{k[0:10]}")

r = np.arange(0,len(corpus_small))
comb = list(combinations(r, 2))
jaccard_similarity(corpus_small, comb)

d1 = open('/content/drive/MyDrive/Spring2023/CS4140/D1.txt', 'r')
d2 = open('/content/drive/MyDrive/Spring2023/CS4140/D2.txt', 'r')

corpus_docs = [d1.read(), d2.read()]
kgrams_hash = []

for c in corpus_docs:
  c_lst = [c]
  kgrams_hash.append(n_grams_char(c_lst, 3))

k_lst = [20, 60, 150, 300, 450, 600, 750, 800, 1000]
for k in k_lst:
  start = time.time()
  fast_mh_a = min_hashing(kgrams_hash[0], k) 
  fast_mh_b = min_hashing(kgrams_hash[1], k) 
  jh = jaccard_hashing(fast_mh_a[0], fast_mh_b[0], k)
  end = time.time()
  print(f"jaccard hashing for k = {k} : {jh}")
  print("duration = ", end-start)

length 2-gram char D0: 333
[' "' ' (' ' /' ' 1' ' 3' ' 4' ' a' ' b' ' c' ' d']
length 2-gram char D1: 461
[' "' ' (' ' -' ' /' ' 1' ' 2' ' a' ' b' ' c' ' d']
length 2-gram char D2: 401
[' "' ' (' ' /' ' 1' ' 2' ' a' ' b' ' c' ' d' ' e']
length 2-gram char D3: 154
[' 1' ' 4' ' a' ' b' ' c' ' d' ' e' ' f' ' g' ' h']


length 3-gram char D0: 933
[' "w' ' (a' ' (g' ' />' ' 10' ' 30' ' 4/' ' a ' ' ab' ' ac']
length 3-gram char D1: 1714
[' "a' ' "b' ' "d' ' "e' ' "g' ' "h' ' "m' ' "s' ' ("' ' (a']
length 3-gram char D2: 1428
[' "c' ' "l' ' "o' ' "p' ' "r' ' ("' ' (i' ' (s' ' (t' ' />']
length 3-gram char D3: 256
[' 1 ' ' 40' ' a ' ' af' ' al' ' an' ' at' ' av' ' ba' ' be']


length 2-gram word D0: 349
['106 minutes' '30 minutes' '4 10' 'a beautiful' 'a complete' 'a girl'
 'a lot' 'a nice' 'a pretty' 'a promising']
length 2-gram word D1: 908
['1930s among' '1937 and' '1937 but' '20th century' 'a bad' 'a bit'
 'a brawl' 'a break' 'a conclusion' 'a contest']
length 2-gram word D2: 596
['1960 s'